In [1]:
import numpy as np
import pandas as pd

from tkinter import *
from tkinter import ttk
from tkinter import messagebox
import tkinter.font as tkFont

from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

from Model import *

In [ ]:
def genLblTxt(text):
    lbl, txt = StringVar(), Entry(Top)
    lbl.set(text)
    Label(Top, textvariable=lbl).pack(padx=5, pady=5)
    txt.pack(padx=5, pady=5)
    return txt

def getter():
    return (
        int(txtEnt[0].get()),
        [int(nn) for nn in (txtEnt[1].get()).split()],
        float(txtEnt[2].get()),
        int(txtEnt[3].get()),
        int(txtEnt[4].get()),
        txtEnt[5].get()
    )

---

In [8]:
def modelRUN():
    df = pd.read_csv('penguins.csv')
    df['gender'] = LabelEncoder().fit_transform(df['gender'])    
    df = pd.DataFrame(ColumnTransformer([(df.columns[0], OneHotEncoder(),
                        [df.columns.get_loc(df.columns[0])])],
                      remainder='passthrough').fit_transform(df),
                      columns=['C1', 'C2', 'C3', 'X1', 'X2', 'X3', 'X4', 'X5'])
    
    try: layers, nn, eta, epochs, bias, activeFn = getter()
    except: layers, nn, eta, epochs, bias, activeFn = 2, (8, 4), 0.01, 100, 1, 'Sigmoid'
    
    df.insert(loc=3, column='bias', value=[bias for _ in range(len(df))])
        
    C1, C2, C3 = df[:50].sample(frac=1), df[50:100].sample(frac=1), df[100:].sample(frac=1)
    trainData = (pd.concat([C1[:30], C2[:30], C3[:30]])).to_numpy()
    testData = (pd.concat([C1[30:], C2[30:], C3[30:]])).to_numpy()
    
    inMat, outMat = np.zeros([len(trainData), 5+1]), np.zeros([len(trainData), 3])
    for i in range(len(trainData)):
        outMat[i], inMat[i] = trainData[i][0:3], trainData[i][3:]
    
    weights = list()
    weights.append(np.random.randn(5+1, nn[0]))
    for i in range(layers-1):
        weights.append(np.random.randn(nn[i], nn[i+1]))
    weights.append(np.random.randn(nn[-1], 3))
    
    weights = BackPropagationAlgo(inMat, outMat, weights, activeFn,
                                            epochs, eta, layers, nn, bias)
    
    print(weights)
    
modelRUN()

X:\Jupyter\Penguins-Classifier\Back Propagation Multilayer\Model.py:6: RuntimeWarning: overflow encountered in exp
  return 1/(1 + np.exp(-val))


[array([[ 0.23775773,  1.41861119, -0.4585735 , -0.82874777, -0.30440456,
         1.61811971,  2.12739256, -0.75831054],
       [-1.17058099, -0.68453305, -0.17410893, -0.14315026, -0.14730625,
         1.62037157,  0.272571  , -0.77346396],
       [ 1.21280066,  0.8704455 , -0.07098573, -1.72761869,  1.21714468,
        -0.5287333 , -1.42173584, -0.98833423],
       [ 0.76878804,  1.31146526,  2.2306769 , -0.41607252, -0.33376053,
         0.01963669,  0.87720474,  0.85497087],
       [ 0.10085809, -0.0296934 , -1.49983507, -0.09590498, -0.28746655,
        -1.15412691, -0.69650089, -1.54243223],
       [ 0.30968978,  0.01504548,  0.85673324, -0.38649829,  0.42883734,
        -0.71177193, -0.18949447, -2.08119346]]), array([[-0.17878126,  1.85730309, -0.24693459,  0.50083784],
       [ 0.50515129, -1.8751756 ,  0.36724719,  0.19622601],
       [ 0.58320475,  0.66945701,  0.51545583, -0.05609377],
       [-0.18140254,  0.93503535,  0.24094891, -1.19032608],
       [-0.0491922 ,  0.115

In [ ]:
def PredictFn():
    for i in range(len(txtEnt)):
        if txtEnt[i].get() in ('', 'Activation Function >>'):
            messagebox.showerror(title="error", message="Insert the missing inputs", parent=Top)
            break
    else:
        #print(getter())
        modelRUN()

---

In [ ]:
Top = Tk()
Top.geometry('300x400')
Top.title('Back Propagation Algo')
Top.resizable(False, False)

txtEnt = [0]*6
txtEnt[0] = (genLblTxt('No. of Hidden Layers'))
txtEnt[1] = (genLblTxt('No. of Neurons\n(separate by space)'))
txtEnt[2] = (genLblTxt('Learning Rate Value'))
txtEnt[3] = (genLblTxt('No. of Epochs'))

txtEnt[4] = IntVar()
cb_obj = Checkbutton(Top, text='Add Bias', variable=txtEnt[4], onvalue=1, offvalue=0)
cb_obj.pack()

txtEnt[5] = ttk.Combobox(Top, textvariable=StringVar(), state='readonly', values=('Sigmoid', 'TanH'))
txtEnt[5].set('Activation Function >>')
txtEnt[5].pack(padx=5, pady=5)

btn_submit = Button(Top, text='Run', width=10, command=PredictFn, bg='green', fg='yellow')
btn_submit.pack(pady=25)

Top.mainloop()